### Assignment 3

#### Step 6 isk Management

Integrate risk management techniques into each strategy to control position sizing, stop-loss levels, and profit-taking mechanisms. This step is crucial to avoid excessive losses and optimize risk-adjusted returns.




In [ ]:
'''
This block only installs the necessary dependencies onto the system. If the libs are already installed, then no need to execute this command.

Known issues 
- The pyfolio lib only works with pandas version < 2.0.0, therefore the highest version below 2.0.0 ( i.e. 1.5.3 ) is used

'''
%pip install pandas==1.5.3
%pip install yfinance==0.2.37
%pip install pyfolio==0.9.2
%pip install empyrical==0.5.5
%pip install pydantic==2.6.3
%pip install matplotlib==3.8.3
%pip install ta==0.11.0


In [ ]:
# import the libs
import pandas as pd
import pandas as pd
import yfinance as yf
import pyfolio as pf
import datetime as dt
import empyrical as em
import ta
import numpy as np
from pydantic import BaseModel
import warnings
from scipy.stats import norm

# Ignore printing all warnings
warnings.filterwarnings('ignore')

In [ ]:
class Settings( BaseModel ):
    period   : str = "10y"
    interval : str = "1d"
    tmp_folder: str = 'tmp'
    yahoo_ticker_ext: str = ".NS"
    risk_free_rate: float = 0.067188
    days_in_year: int = 252

setting = Settings( )

# dataframes to hold the metrics related data
descriptive_stats = pd.DataFrame( ) 
custom_metrics = pd.DataFrame( )
pyfolio_metrics = pd.DataFrame( )

def get_performance_metrics( return_data ):

    # calcuate the annual ration
    annual_return       = np.mean( return_data ) * setting.days_in_year
    
    # calculate annual volatility ratio
    annual_volatility   = np.std(return_data) * np.sqrt( setting.days_in_year )
    
    # calculate sharpe ratio
    sharpe_ratio        = (annual_return - setting.risk_free_rate) / annual_volatility

    # calculate rolling max window
    rolling_max = return_data.cummax()

    # daily drawdown
    daily_drawdown = return_data / rolling_max - 1.0
    
    max_drawdown = daily_drawdown.cummin().min()

    calmar_ratio = (annual_return - setting.risk_free_rate ) / abs(max_drawdown)

    daily_var = norm.ppf(1-0.05, np.mean(return_data), np.std(return_data))

    negative_std = np.std( return_data [return_data < 0])
    sortino_ratio = (annual_return -setting.risk_free_rate)/(negative_std * np.sqrt(252))

    positive_returns_sum = np.sum( return_data[return_data > 0])
    negative_returns_sum = np.sum( return_data[return_data < 0])
    omega_ratio = positive_returns_sum / abs(negative_returns_sum)

    tail_ratio = abs(np.percentile(return_data.dropna(), 95)) / abs(np.percentile( return_data.dropna(), 5))

    metrics = {
        'annual_return': annual_return,
        'annual_volatility': annual_volatility,
        'sharpe_ratio' : sharpe_ratio,
        'max_drawdown': max_drawdown,
        'calmar_ratio': calmar_ratio,
        'daily_value_at_risk': daily_var,
        'sortino_ratio': sortino_ratio,
        'omega_ratio': omega_ratio,
        'tail_ratio': tail_ratio
    }

    return metrics

# download the data from yfinance
stock_symbol = "ICICIBANK.NS" 
drop_column_list =  ['Open', 'Volume', 'Stock Splits', 'Dividends']

ticker = yf.Ticker( stock_symbol )
ticker_df = ticker.history( period=setting.period, interval=setting.interval )
ticker_df.dropna(inplace = True)
ticker_df['daily_returns'] = np.log(ticker_df['Close']/ticker_df['Close'].shift(1))

ticker_df.drop( drop_column_list ,axis=1, inplace=True, errors='ignore')

ticker_df.head( )